### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load
file_to_load = "purchase_data.csv"

In [2]:
class HeroesOfPymoli():
    '''
    This class contains answers to all of the segments in this notebook.
    - self.player_count() answers the Player Count segment.
    - self.purchasing_analysis() answers the Purchasing Analysis (Total) segment.
    - self.gender_demographics() answers the Gender Demographics segment.
    - self.gender_purchasing_analysis() answers the Purchasing Analysis (Gender) segment.
    - self.age_demographics() answers the Age Demographics segment.
    - self.age_purchasing_analysis() answers the Purchasing Analysis (Age) segment.
    - self.top_spenders() answers the Top Spenders segment.
    - self.item_stats(sort_by) answers the Most Popular Items and Most Profitable Items segments depending on the sort_by parameter.
        - sort_by can be set to either 'Purchase Count' or 'Total Purchase Value'.
    '''
    def __init__(self, file_to_load):
        # Read Purchasing File and Store into Pandas DataFrame
        self.purchase_data = pd.read_csv(file_to_load)
    
    def player_count(self):
        '''
        Returns dataframe containing the total number of players
        '''
        # List of all unique players appearing in the purchasing data
        unique_players = list(self.purchase_data['SN'].unique())
        number_of_players = len(unique_players)
        num_of_players_df = pd.DataFrame([number_of_players], columns = ['Total Players'])
        return num_of_players_df
        
    def purchasing_analysis(self):
        '''
        Basic calculations to obtain:
        - number of unique items
        - avg. purchase price for all purchases
        - avg. price for all items
        - purchase count
        - total revenue
        - avg. purchase total per person
        
        Returns summary dataframe containing these numbers.
        '''
        
        column_names = ['Number of Unique Items', 'Average Purchase', 'Average Price', 'Number of Purchases', 'Total Revenue', 'Average Purchase Total per Person']
        
        # Number of Unique Items
        unique_items = list(self.purchase_data['Item ID'].unique())
        number_of_items = len(unique_items)
        
        # Average Purchase 
        avg_purchase = self.purchase_data['Price'].mean()
        avg_purchase = '$' + '{:.2f}'.format(round(avg_purchase, 2))
        
        # Average Price
        unique_items_df = self.purchase_data.drop_duplicates(subset=['Item ID'])
        avg_item_price = unique_items_df['Price'].mean()
        avg_item_price = '$' + '{:.2f}'.format(round(avg_item_price, 2))
        
        # Purchase Count
        purchase_count, _ = self.purchase_data.shape
        
        # Total Revenue
        total_revenue = self.purchase_data['Price'].sum()
        total_revenue = '$' + '{:.2f}'.format(round(total_revenue, 2))
        
        # Average Purchase Total per Person
        player_df = self.purchase_data.copy()
        player_df['Price'] = player_df.groupby(['SN'])['Price'].transform('sum')
        player_df.drop_duplicates(subset=['SN'], inplace=True)
        avg_purch_total = player_df['Price'].mean()
        avg_purch_total = '$' + '{:.2f}'.format(round(avg_purch_total, 2))
        
        purchasing_analysis_df = pd.DataFrame(list(zip([number_of_items], [avg_purchase], [avg_item_price], [purchase_count], [total_revenue], [avg_purch_total])),
                                              columns = column_names)
        return purchasing_analysis_df
        
    def gender_demographics(self):
        '''
        Returns dataframe containing:
        - Percentage and Count of Male Players
        - Percentage and Count of Female Players
        - Percentage and Count of Other / Non-Disclosed
        '''
        column_names = ['index', 'Total Count', 'Percentage of Players']
        index = ['Male', 'Female', 'Other/Non-Disclosed']
        
        unique_players_df = self.purchase_data.drop_duplicates(subset=['SN'])
        number_of_players, _ = unique_players_df.shape
        
        # Counts of each gender
        gender_counts = unique_players_df['Gender'].value_counts()
        number_of_males = gender_counts.loc['Male']
        number_of_females = gender_counts.loc['Female']
        number_of_others = number_of_players - number_of_males - number_of_females
        
        gender_counts = [number_of_males, number_of_females, number_of_others]
        gender_percentages = []
        
        # Calculating percentages for each gender
        for count in gender_counts:
            figure = round(count * 100 / number_of_players, 2)
            percent = str(figure) + '%'
            gender_percentages.append(percent)
        
        gender_demographics_df = pd.DataFrame(list(zip(index, gender_counts, gender_percentages)), 
                                             columns = column_names)
        gender_demographics_df.set_index(['index'], inplace=True)
        gender_demographics_df.index.name = None
        
        return gender_demographics_df
        
    def gender_purchasing_analysis(self):
        '''
        Basic calculations by gender:
        - avg. purchase price for all purchases
        - purchase count
        - avg. purchase total per person
        - total purchase value
        
        Returns summary dataframe containing these numbers.
        '''
        column_names = ['Gender', 'Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Average Purchase Total per Person']
        gender = ['Male', 'Female', 'Other / Non-Disclosed']
        
        male_df = self.purchase_data[self.purchase_data['Gender'] == 'Male']
        female_df = self.purchase_data[self.purchase_data['Gender'] == 'Female']
        other_df = self.purchase_data[(self.purchase_data['Gender'] != 'Male') & (self.purchase_data['Gender'] != 'Female')]
        
        dataframes = [male_df, female_df, other_df]
        
        purchase_counts = []
        avg_purch_prices = []
        total_purchase_values = []
        avg_purch_totals = []
        
        for dataframe in dataframes:
            # Purchase count
            purchase_count, _ = dataframe.shape
            purchase_counts.append(purchase_count)
            
            # Average purchase price for all purchases
            avg_purch_price = dataframe['Price'].mean()
            
            avg_purch_price = '$' + '{:.2f}'.format(round(avg_purch_price, 2))
            avg_purch_prices.append(avg_purch_price)
            
            # Total purchase value
            total_purchase_value = dataframe['Price'].sum()
            total_purchase_value = '$' + '{:.2f}'.format(round(total_purchase_value, 2))
            total_purchase_values.append(total_purchase_value)
            
            # Average purchase total per person
            dataframe['Price'] = dataframe.groupby(['SN'])['Price'].transform('sum')
            dataframe.drop_duplicates(subset=['SN'], inplace=True)
            avg_purch_total = dataframe['Price'].mean()
            avg_purch_total = '$' + '{:.2f}'.format(round(avg_purch_total, 2))
            avg_purch_totals.append(avg_purch_total)
        
        gender_purch_analysis_df = pd.DataFrame(list(zip(gender, purchase_counts, avg_purch_prices, total_purchase_values, avg_purch_totals)),
                                               columns = column_names)
        gender_purch_analysis_df.set_index(['Gender'], inplace=True)
        
        return gender_purch_analysis_df
        
    def age_demographics(self):
        '''
        Establishes bins for ages and categorizes players using the age bins.
        For each bin, calculates
        - the number of players in that bin
        - the percentage of all players that age group contains, rounded to two decimal points
        
        Returns summary dataframe containing these numbers.
        '''
        column_names = ['Age Group', 'Total Count', 'Percentage of Players']
        
        bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
        age_groups = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
        
        unique_players_df = self.purchase_data.drop_duplicates(subset=['SN'])
        number_of_players, _ = unique_players_df.shape
        
        # Assigning bins to each player
        unique_players_df['Age Group'] = pd.cut(x=unique_players_df['Age'], bins=bins, labels=age_groups)
        
        age_group_counts = unique_players_df['Age Group'].value_counts()
        
        player_counts = []
        percentages = []
        
        for age_group in age_groups:
            # Number of players in each bin
            age_group_count = age_group_counts.loc[age_group]
            player_counts.append(age_group_count)
            
            # Percentage of players in each bin
            percent = age_group_count * 100 / number_of_players
            percent = '{:.2f}'.format(round(percent, 2)) + '%'
            percentages.append(percent)
        
        age_demographics_df = pd.DataFrame(list(zip(age_groups, player_counts, percentages)),
                                          columns = column_names)
        age_demographics_df.set_index(['Age Group'], inplace=True)
        age_demographics_df.index.name = None
        
        return age_demographics_df
        
    def age_purchasing_analysis(self):
        '''
        Based on the age bins categorization of players, bins the purchase data by age bins.
        Runs basic calculations to obtain:
        - purchase count
        - avg. purchase price
        - total purchase value
        - avg. purchase total per person
        
        Returns a summary data frame containing these numbers.
        '''
        column_names = ['Age Ranges', 'Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Average Total Purchase per Person']
        
        bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
        age_groups = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
        
        purchase_data = self.purchase_data.copy()
        
        purchase_data['Age Ranges'] = pd.cut(x=purchase_data['Age'], bins=bins, labels=age_groups)
        
        purchase_counts = []
        avg_purch_prices = []
        total_purchase_values = []
        avg_purch_totals = []
        
        for age_group in age_groups:
            dataframe = purchase_data[purchase_data['Age Ranges'] == age_group]
            
            # Purchase count
            purchase_count, _ = dataframe.shape
            purchase_counts.append(purchase_count)
            
            # Average purchase price for all purchases
            avg_purch_price = dataframe['Price'].mean()
            
            avg_purch_price = '$' + '{:.2f}'.format(round(avg_purch_price, 2))
            avg_purch_prices.append(avg_purch_price)
            
            # Total purchase value
            total_purchase_value = dataframe['Price'].sum()
            total_purchase_value = '$' + '{:.2f}'.format(round(total_purchase_value, 2))
            total_purchase_values.append(total_purchase_value)
            
            # Average purchase total per person
            dataframe['Price'] = dataframe.groupby(['SN'])['Price'].transform('sum')
            dataframe.drop_duplicates(subset=['SN'], inplace=True)
            avg_purch_total = dataframe['Price'].mean()
            avg_purch_total = '$' + '{:.2f}'.format(round(avg_purch_total, 2))
            avg_purch_totals.append(avg_purch_total)
        
        age_purch_analysis_df = pd.DataFrame(list(zip(age_groups, purchase_counts, avg_purch_prices, total_purchase_values, avg_purch_totals)),
                                               columns = column_names)
        age_purch_analysis_df.set_index(['Age Ranges'], inplace=True)
        
        return age_purch_analysis_df
        
    def top_spenders(self):
        '''
        Runs basic calculations to obtain:
        - purchase count
        - average purchase price
        - total purchase value
        for each player.
        
        Returns a dataframe containing this data ordered by total purchase value.
        '''
        column_names = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
        
        spending_data = self.purchase_data.copy()
        # Purchase Count
        purchase_count_df = spending_data['SN'].value_counts().rename_axis('SN').reset_index(name='Purchase Count')
        purchase_count_df.set_index(['SN'], inplace=True)
        
        # Total Purchase Value
        spending_data['Total Purchase Value'] = spending_data.groupby(['SN'])['Price'].transform('sum')
        # Average Purchase Price
        spending_data['Average Purchase Price'] = spending_data.groupby(['SN'])['Price'].transform('mean')
        spending_data.drop_duplicates(subset=['SN'], inplace=True)
        spending_data.drop(spending_data.columns.difference(['SN','Total Purchase Value', 'Average Purchase Price']), 1, inplace=True)
        spending_data.set_index(['SN'], inplace=True)
        
        top_spenders = pd.concat([spending_data, purchase_count_df], axis=1, join='inner')
        
        top_spenders.sort_values(by=['Total Purchase Value'], ascending=False, inplace=True)
        
        top_spenders = top_spenders[column_names]
        top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].apply(lambda x: '$' + '{:.2f}'.format(round(x, 2)))
        top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].apply(lambda x: '$' + '{:.2f}'.format(round(x, 2)))
        
        return top_spenders
    
    def item_stats(self, sort_by):
        '''
        Based on the Item ID, Item Name, and Item Price columns. Groups by Item ID and Item Name.
        Runs basic calculations to obtain:
        - purchase count
        - average item price
        - total purchase value
        for each Item ID.
        
        Depending on sort_by parameter, returns a dataframe containing this data ordered by either
        purchase count or total purchase value.
        '''
        column_names = ['Purchase Count', 'Item Price', 'Total Purchase Value']
        
        item_data = self.purchase_data.copy()
        
        # Purchase Count
        purchase_count_df = item_data['Item ID'].value_counts().rename_axis('Item ID').reset_index(name='Purchase Count')
        purchase_count_df.set_index(['Item ID'], inplace=True)
        
        # Average Item Price for each item
        item_data['Total Purchase Value'] = item_data.groupby(['Item ID'])['Price'].transform('sum')
        # Total Purchase Value for each item
        item_data['Item Price'] = item_data.groupby(['Item ID'])['Price'].transform('mean')
        item_data.drop_duplicates(subset=['Item ID'], inplace=True)
        
        item_data.drop(item_data.columns.difference(['Item ID','Total Purchase Value', 'Item Name', 'Item Price']), 1, inplace=True)
        item_data.set_index(['Item ID'], inplace=True)
        
        item_stats = pd.concat([item_data, purchase_count_df], axis=1, join='inner')
        
        item_stats.set_index(['Item Name'], append=True, inplace=True)
        
        item_stats.sort_values(by=[sort_by], ascending=False, inplace=True)
        
        item_stats = item_stats[column_names]
        item_stats['Item Price'] = item_stats['Item Price'].apply(lambda x: '$' + '{:.2f}'.format(round(x, 2)))
        item_stats['Total Purchase Value'] = item_stats['Total Purchase Value'].apply(lambda x: '$' + '{:.2f}'.format(round(x, 2)))
        
        return item_stats

In [3]:
answers = HeroesOfPymoli(file_to_load)

## Player Count

* Display the total number of players


In [4]:
player_count = answers.player_count()
player_count

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
purchasing_analysis_df = answers.purchasing_analysis()
purchasing_analysis_df

,Number of Unique Items,Average Purchase,Average Price,Number of Purchases,Total Revenue,Average Purchase Total per Person
0,179,$3.05,$3.04,780,$2379.77,$4.13


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
gender_demographics_df = answers.gender_demographics()
gender_demographics_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
gender_purchasing_analysis_df = answers.gender_purchasing_analysis()
gender_purchasing_analysis_df

<ipython-input-2-430848694889>:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Price'] = dataframe.groupby(['SN'])['Price'].transform('sum')
<ipython-input-2-430848694889>:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.drop_duplicates(subset=['SN'], inplace=True)


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Gender,,,,
Male,652,$3.02,$1967.64,$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [8]:
age_demographics_df = answers.age_demographics()
age_demographics_df

<ipython-input-2-430848694889>:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_players_df['Age Group'] = pd.cut(x=unique_players_df['Age'], bins=bins, labels=age_groups)


,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
age_purchasing_analysis_df = answers.age_purchasing_analysis()
age_purchasing_analysis_df

<ipython-input-2-430848694889>:248: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Price'] = dataframe.groupby(['SN'])['Price'].transform('sum')
<ipython-input-2-430848694889>:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.drop_duplicates(subset=['SN'], inplace=True)


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
top_spenders_df = answers.top_spenders()
top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
popular_items_df = answers.item_stats(sort_by='Purchase Count')
popular_items_df.head(10)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [12]:
profitable_items_df = answers.item_stats(sort_by='Total Purchase Value')
profitable_items_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
